<a href="https://colab.research.google.com/github/iwatake2222/colaboratory_study/blob/master/object_detection/beer_detection/beer_detection_train_ssd_mobilenet_v2_quantized_300x300.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object detection model training
- dataset: beer
- pretrained model: ssd_mobilenet_v2_quantized_300x300_coco
  - https://github.com/tensorflow/models/blob/master/research/object_detection/samples/configs/ssd_mobilenet_v2_quantized_300x300_coco.config
    - modify and upload to google drive
  - http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz
    - can be found here: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md
- note
  - need to decrease the number of `batch_size` in config file, otherwise OOM occurs
  - find `### MODIFIED ###` in the config file

## Preparation

In [0]:
from google.colab import drive 
drive.mount('/content/drive')
!ls

TRAIN_DIR = "/content/drive/My Drive/training/beer_detection/ssd_mobilenet_v2_quantized_300x300_20191230"

In [0]:
%tensorflow_version 1.x
# restart runtime
# !pip install tensorflow
import tensorflow as tf
print(tf.__version__)
!pip install pycocotools
!pip list

!git clone --depth 1 https://github.com/tensorflow/models
%cd models/research/
!protoc object_detection/protos/*.proto --python_out=.
!pip install .
%cd /content

1.15.0
Package                  Version    
------------------------ -----------
absl-py                  0.8.1      
alabaster                0.7.12     
albumentations           0.1.12     
altair                   4.0.0      
asgiref                  3.2.3      
astor                    0.8.1      
astropy                  3.0.5      
atari-py                 0.2.6      
atomicwrites             1.3.0      
attrs                    19.3.0     
audioread                2.1.8      
autograd                 1.3        
Babel                    2.7.0      
backcall                 0.1.0      
backports.tempfile       1.0        
backports.weakref        1.0.post1  
beautifulsoup4           4.6.3      
bleach                   3.1.0      
blis                     0.2.4      
bokeh                    1.0.4      
boto                     2.49.0     
boto3                    1.10.40    
botocore                 1.13.40    
Bottleneck               1.3.1      
branca                   0.3.1 

## Get pretrained model
need this only once

In [0]:
'''
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz
!tar -xvf ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz
!mkdir -p "/content/drive/My Drive/model_original/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03"
!cp ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/model.ckpt.* "/content/drive/My Drive/model_original/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/"
'''

model.ckpt.data-00000-of-00001	model.ckpt.index  model.ckpt.meta


## Minor modification
Modify python path

In [0]:
%set_env PYTHONPATH=$PYTHONPATH:models/research:models/research/slim
!echo $PYTHONPATH
!python models/research/object_detection/builders/model_builder_test.py

env: PYTHONPATH=$PYTHONPATH:models/research:models/research/slim
$PYTHONPATH:models/research:models/research/slim
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTest.test_create_experimental_model
[       OK ] ModelBuilderTest.test_create_experimental_model
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[       OK ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[       OK ] ModelBuilderTest.test_create_faster_rcnn_models_from_co

## Training
### config file
ssd_mobilenet_v2_quantized_300x300_coco_beer.config

```
# Quantized trained SSD with Mobilenet v2 on MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    ### MODIFIED ###
    num_classes: 10
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 3.0
        aspect_ratios: 0.3333
      }
    }
    image_resizer {
      fixed_shape_resizer {
        height: 300
        width: 300
      }
    }
    box_predictor {
      convolutional_box_predictor {
        min_depth: 0
        max_depth: 0
        num_layers_before_predictor: 0
        use_dropout: false
        dropout_keep_probability: 0.8
        kernel_size: 1
        box_code_size: 4
        apply_sigmoid_to_scores: false
        conv_hyperparams {
          activation: RELU_6,
          regularizer {
            l2_regularizer {
              weight: 0.00004
            }
          }
          initializer {
            truncated_normal_initializer {
              stddev: 0.03
              mean: 0.0
            }
          }
          batch_norm {
            train: true,
            scale: true,
            center: true,
            decay: 0.9997,
            epsilon: 0.001,
          }
        }
      }
    }
    feature_extractor {
      type: 'ssd_mobilenet_v2'
      min_depth: 16
      depth_multiplier: 1.0
      conv_hyperparams {
        activation: RELU_6,
        regularizer {
          l2_regularizer {
            weight: 0.00004
          }
        }
        initializer {
          truncated_normal_initializer {
            stddev: 0.03
            mean: 0.0
          }
        }
        batch_norm {
          train: true,
          scale: true,
          center: true,
          decay: 0.9997,
          epsilon: 0.001,
        }
      }
    }
    loss {
      classification_loss {
        weighted_sigmoid {
        }
      }
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      hard_example_miner {
        num_hard_examples: 3000
        iou_threshold: 0.99
        loss_type: CLASSIFICATION
        max_negatives_per_positive: 3
        min_negatives_per_image: 3
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    normalize_loss_by_num_matches: true
    post_processing {
      batch_non_max_suppression {
        score_threshold: 1e-8
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SIGMOID
    }
  }
}

train_config: {
  ### MODIFIED ###
  batch_size: 10
  optimizer {
    rms_prop_optimizer: {
      learning_rate: {
        exponential_decay_learning_rate {
          initial_learning_rate: 0.004
          decay_steps: 800720
          decay_factor: 0.95
        }
      }
      momentum_optimizer_value: 0.9
      decay: 0.9
      epsilon: 1.0
    }
  }
  ### MODIFIED ###
  fine_tune_checkpoint: "/content/drive/My Drive/model_original/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/model.ckpt"
  fine_tune_checkpoint_type:  "detection"
  # Note: The below line limits the training process to 200K steps, which we
  # empirically found to be sufficient enough to train the pets dataset. This
  # effectively bypasses the learning rate schedule (the learning rate will
  # never decay). Remove the below line to train indefinitely.
  num_steps: 200000
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    ssd_random_crop {
    }
  }
}

train_input_reader: {
  ### MODIFIED ###
  tf_record_input_reader {
    input_path: "/content/drive/My Drive/training/beer_detection/data/train/*.tfrecord"
  }
  ### MODIFIED ###
  label_map_path: "/content/drive/My Drive/datasets/beer/tf_label_map.pbtxt"
}

eval_config: {
  num_examples: 8000
  # Note: The below line limits the evaluation process to 10 evaluations.
  # Remove the below line to evaluate indefinitely.
  max_evals: 10
}

eval_input_reader: {
  ### MODIFIED ###
  tf_record_input_reader {
    input_path: "/content/drive/My Drive/training/beer_detection/data/eval/*.tfrecord"
  }
  ### MODIFIED ###
  label_map_path: "/content/drive/My Drive/datasets/beer/tf_label_map.pbtxt"
  shuffle: false
  num_readers: 1
}

graph_rewriter {
  quantization {
    delay: 48000
    weight_bits: 8
    activation_bits: 8
  }
}

```

In [8]:
!python models/research/object_detection/model_main.py \
    --pipeline_config_path="$TRAIN_DIR/ssd_mobilenet_v2_quantized_300x300_coco_beer.config" \
    --model_dir="$TRAIN_DIR" \
    --num_train_steps=10000 \
    --num_eval_steps=2000 \
    --alsologtostderr


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W1230 07:30:14.385492 140603651950464 module_wrapper.py:139] From /content/models/research/object_detection/utils/config_util.py:102: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1230 07:30:14.889451 140603651950464 module_wrapper.py:139] From /content/models/research/object_detection/model_lib.py:628: The name tf.logging.warning is deprecated. Please use tf.compat.v1.logging.warning instead.

W1230 07:30:14.889662 140603651950464 model_lib.py:629] Forced number of epochs for all eval validations to be 1.

W1230 07:30:14.889773 140603651950464 module_wrapper.py:139] From /content/models/res

## Export graph

In [0]:
# this doesn't work
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path="$TRAIN_DIR/ssd_mobilenet_v2_quantized_300x300_coco_beer.config" \
    --trained_checkpoint_prefix="$TRAIN_DIR/model.ckpt-10000" \
    --output_directory="$TRAIN_DIR/exported_graphs"


In [13]:
# Use this instead
!python models/research/object_detection/export_tflite_ssd_graph.py	\
    --input_type=image_tensor \
    --pipeline_config_path="$TRAIN_DIR/ssd_mobilenet_v2_quantized_300x300_coco_beer.config" \
    --trained_checkpoint_prefix="$TRAIN_DIR/model.ckpt-10000" \
    --output_directory="$TRAIN_DIR/exported_graphs2" \
    --add_postprocessing_op=true 


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W1230 09:30:35.169306 140294663522176 module_wrapper.py:139] From models/research/object_detection/export_tflite_ssd_graph.py:133: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1230 09:30:35.174433 140294663522176 module_wrapper.py:139] From /content/models/research/object_detection/export_tflite_ssd_graph_lib.py:193: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W1230 09:30:35.177208 140294663522176 module_wrapper.py:139] From /content/models/research/object_detection/export_tflite_ssd_graph_lib.py:237: The name tf.placeholder is deprecated. Please use

## Check model structure
to identify input and output nodes

In [16]:
Graph = tf.GraphDef()
File = open(TRAIN_DIR + "/exported_graphs2/tflite_graph.pb","rb")
Graph.ParseFromString(File.read())
for Layer in Graph.node:
  print(Layer.name)


normalized_input_image_tensor
FeatureExtractor/MobilenetV2/MobilenetV2/input
FeatureExtractor/MobilenetV2/Conv/weights
FeatureExtractor/MobilenetV2/Conv/weights/read
FeatureExtractor/MobilenetV2/Conv/BatchNorm/gamma
FeatureExtractor/MobilenetV2/Conv/BatchNorm/gamma/read
FeatureExtractor/MobilenetV2/Conv/BatchNorm/beta
FeatureExtractor/MobilenetV2/Conv/BatchNorm/beta/read
FeatureExtractor/MobilenetV2/Conv/BatchNorm/moving_mean
FeatureExtractor/MobilenetV2/Conv/BatchNorm/moving_mean/read
FeatureExtractor/MobilenetV2/Conv/BatchNorm/moving_variance
FeatureExtractor/MobilenetV2/Conv/BatchNorm/moving_variance/read
FeatureExtractor/MobilenetV2/Conv/Relu6
FeatureExtractor/MobilenetV2/expanded_conv/input
FeatureExtractor/MobilenetV2/expanded_conv/depthwise/depthwise_weights
FeatureExtractor/MobilenetV2/expanded_conv/depthwise/depthwise_weights/read
FeatureExtractor/MobilenetV2/expanded_conv/depthwise/BatchNorm/gamma
FeatureExtractor/MobilenetV2/expanded_conv/depthwise/BatchNorm/gamma/read
Featu

## Convert to tflite


In [66]:
input_arrays = ["normalized_input_image_tensor"]
converter = tf.lite.TFLiteConverter.from_frozen_graph(
  TRAIN_DIR + "/exported_graphs2/tflite_graph.pb",
  input_arrays,
  ["TFLite_Detection_PostProcess", "TFLite_Detection_PostProcess:1", "TFLite_Detection_PostProcess:2", "TFLite_Detection_PostProcess:3"],
  input_shapes={input_arrays[0]:[1,300,300,3]}
)

#converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.allow_custom_ops=True
converter.inference_input_type = tf.lite.constants.QUANTIZED_UINT8
# converter.inference_output_type = tf.lite.constants.QUANTIZED_UINT8
converter.quantized_input_stats = {input_arrays[0] : (128., 127.)}  # mean, std_dev ((quantized_input_value - mean_value) / std_dev_value.)
converter.default_ranges_stats = (-10, 10)
converter.inference_type = tf.lite.constants.QUANTIZED_UINT8

tflite_model = converter.convert()
open(TRAIN_DIR + "/beer_mobilenet_v2_quantized_300x300.tflite", "wb").write(tflite_model)


4859376

In [0]:
%%bash 
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
sudo apt-get update
sudo apt-get install edgetpu

In [67]:
!edgetpu_compiler "$TRAIN_DIR/beer_mobilenet_v2_quantized_300x300.tflite"
!cp beer_mobilenet_v2_quantized_300x300_edgetpu.tflite "$TRAIN_DIR"

Edge TPU Compiler version 2.0.267685300

Model compiled successfully in 339 ms.

Input model: /content/drive/My Drive/training/beer_detection/ssd_mobilenet_v2_quantized_300x300_20191230/beer_mobilenet_v2_quantized_300x300.tflite
Input size: 4.63MiB
Output model: beer_mobilenet_v2_quantized_300x300_edgetpu.tflite
Output size: 5.41MiB
On-chip memory available for caching model parameters: 7.62MiB
On-chip memory used for caching model parameters: 5.20MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 99
Operation log: beer_mobilenet_v2_quantized_300x300_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 98
Number of 